# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file = "output_data/cities.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Hermanus,100,ZA,1586634368,87,-34.42,19.23,64.99,10.00
1,Port Alfred,97,ZA,1586634369,91,-33.59,26.89,68.00,7.00
2,Olafsvik,60,IS,1586634369,79,64.89,-23.71,37.24,28.39
3,Berlevåg,79,NO,1586634369,90,70.86,29.09,25.02,3.24
4,Barrow,90,US,1586634369,84,71.29,-156.79,-2.20,11.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# A max temperature lower than 80 degrees but higher than 70.
idealweather_df = weather_df.loc[weather_df["Max Temp"]<80]
idealweather_df = weather_df.loc[weather_df["Max Temp"]>70]
# Wind speed less than 10 mph.
idealweather_df = weather_df.loc[weather_df["Wind Speed"]<10]
#  Zero cloudiness.
idealweather_df = weather_df.loc[weather_df["Cloudiness"]==0]

idealweather_df.dropna

idealweather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
8,Karīmganj,0,IN,1586634370,49,24.87,92.35,72.12,3.20
9,Thinadhoo,0,MV,1586634370,62,0.53,72.93,84.16,4.38
20,Mossamedes,0,AO,1586634349,65,-15.20,12.15,77.70,2.46
27,Jining,0,CN,1586634017,60,35.41,116.58,42.89,7.81
63,Podgorica,0,ME,1586634377,31,42.44,19.26,64.40,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df=idealweather_df


params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 8: Karīmganj.
Closest hotel is Ahar Hotel And Aheli Restaurant.
------------
Retrieving Results for Index 9: Thinadhoo.
Missing field/result... skipping.
------------
Retrieving Results for Index 20: Mossamedes.


C:\Users\koros\AppData\Local\Continuum\miniconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\koros\AppData\Local\Continuum\miniconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel is Chik-Chik Namibe.
------------
Retrieving Results for Index 27: Jining.
Closest hotel is Yunhe Star Hotel.
------------
Retrieving Results for Index 63: Podgorica.
Closest hotel is Hotel Podgorica.
------------
Retrieving Results for Index 78: Aksarka.
Missing field/result... skipping.
------------
Retrieving Results for Index 83: Yerbogachën.
Missing field/result... skipping.
------------
Retrieving Results for Index 84: Adrar.
Missing field/result... skipping.
------------
Retrieving Results for Index 88: Mount Isa.
Closest hotel is ibis Styles Mt Isa Verona.
------------
Retrieving Results for Index 120: Ayagoz.
Closest hotel is Gostinitsa Nur.
------------
Retrieving Results for Index 137: Boguchany.
Closest hotel is Uyut.
------------
Retrieving Results for Index 145: Rawson.
Closest hotel is Hotel Punta León.
------------
Retrieving Results for Index 147: Piaçabuçu.
Closest hotel is Piaçabuçu News.
------------
Retrieving Results for Index 157: Byron Bay.
Closest

In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Assign the marker layer to a variable
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))